In [2]:
import cv2 as cv
import numpy as np

# Question Extraction

In [59]:
def searchImg(haystack_img, needle_img, threshold):
    result = cv.matchTemplate(haystack_img, needle_img, cv.TM_CCOEFF_NORMED)

    threshold = 0.7
    locations = np.array(np.where(result >= threshold))

    # Filtering for when it gets the same square multiple times
    height = needle_img.shape[0]
    diffs = np.diff(locations[0])
    indices = np.where(diffs > height)[0]
    selected_indices = np.concatenate(([0], indices+1))
    locations = locations[:, selected_indices]

    locations = list(zip(*locations[::-1]))

    return locations

def trimWhitespace(image, output_path="trimmed_output.jpg"):

    # Convert to grayscale
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)

    # Apply a binary threshold to detect non-white regions
    _, thresh = cv.threshold(gray, 240, 255, cv.THRESH_BINARY_INV)

    # Find contours of the non-white regions
    contours, _ = cv.findContours(thresh, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    if contours:
        # Get bounding box of the largest contour
        x, y, w, h = cv.boundingRect(np.concatenate(contours))

        # Crop the image using the bounding box
        cropped = image[y:y+h, x:x+w]

        # Save or return the cropped image
        cv.imwrite(output_path, cropped)
        return cropped

def extractRegions(regions, haystack_img, needle_width, needle_height):
    haystack_width = haystack_img.shape[1]
    haystack_height = haystack_img.shape[0]
    
    c = 0
    for reg_start, reg_end in regions:
        top_left = (reg_start[0] + int(1.1*needle_width), reg_start[1] - 4*needle_height//5)
        bottom_right = (reg_start[0] + 3*haystack_width//4, reg_end[1] - needle_height//2)

        # cv.rectangle(haystack_img, top_left, bottom_right, line_color, line_type)

        # Extract the region of interest (ROI) from the original image
        roi = trimWhitespace(haystack_img[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]])
        # Save the extracted rectangle as a new image
        output_roi_path = f"output/extracted_question-{c}.jpg"  # Change the path as needed
        cv.imwrite(output_roi_path, roi)
        c += 1
        
    output_path = "output/matched_result.jpg"  # Change the path as needed
    cv.imwrite(output_path, haystack_img)

In [60]:
haystack_img = cv.imread("data/splitted_pages-jpg/output-001.jpg", cv.IMREAD_COLOR)
haystack_width = haystack_img.shape[1]
haystack_height = haystack_img.shape[0]

needle_img = cv.imread("data/Black_square.jpg", cv.IMREAD_COLOR)
needle_height = needle_img.shape[0]
needle_width = needle_img.shape[1]

questao_img = cv.imread("data/Questao.jpg", cv.IMREAD_UNCHANGED)
questao_img = cv.cvtColor(questao_img, cv.COLOR_BGRA2BGR)

locations_black_square = np.array(searchImg(haystack_img, needle_img, 0.7))
locations_questao = np.array(searchImg(haystack_img, questao_img, 0.7))

locations = locations_black_square + locations_questao

In [61]:
regions = []

for loc_square in locations_black_square:
    for loc_quest in locations_questao:
        if loc_quest[1] > loc_square[1]:
            regions.append([loc_square, loc_quest])
            break

# Adding from the last question to the end of the page
regions.append([locations_black_square[-1], np.array([haystack_width, haystack_height])])

regions

[[array([388, 938]), array([ 355, 1349])],
 [array([ 388, 1630]), array([ 355, 2041])],
 [array([ 388, 2637]), array([2480, 3508])]]

In [62]:
extractRegions(regions, haystack_img, needle_width, needle_height)